# Rewrite Reuse Refactor
## Reproducing Unit Tests using refactored ODOG formulation
This Notebook introduces a new way of formulating the normalization step of the ODOG family of Models, and shows that this formulation is numerically equivalent to the original paper implementation, to establish continuity

In [ ]:
# Third party libraries
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import scipy

# Import local module
import multyscale

In [ ]:
# %% Load example stimulus
stimulus = np.asarray(Image.open("image_source/example_stimulus.png").convert("L"))

# %% Parameters of image
# visual extent, same convention as pyplot:
visextent = (-16, 16, -16, 16)

In [ ]:
# Frontend filterbank of ODOG implementation by Robinson et al. (2007)
filterbank = multyscale.filterbank.RHS2007(filtershape=stimulus.shape, visextent=visextent)

# Filter the example stimulus
filters_output = filterbank.apply(stimulus)

This preamble defines a filterbank output which is not already normalized by any function. The Traditional ODOG normalization function is multyscale/models.py ODOG, shown below

In [ ]:
def normalize_outputs(self, filters_output):
        # TODO: docstring
        normalizers = self.normalizers(filters_output)

        normalizer_RMS = self.normalizers_to_RMS(normalizers)

        normalized_outputs = np.ndarray(filters_output.shape)
        for o, s in np.ndindex(filters_output.shape[:2]):
            normalized_outputs[o, s] = filters_output[o, s] / normalizer_RMS[o, s]

        return normalized_outputs


We're comparing this to a new implementation which replaces the math of the original function by a more general form.  
The traditional ODOG formulation looks like this:

$$ODOG: \frac{f_{o^*,s^*}}{\sqrt{\frac{1}{1024^2}\sum_{x,y=0}^{1024,1024}(\sum_{o,s=1}^{O, S}{w_{o,s} f_{o,s})^2}}}$$
where $w_{o,s} =   \begin{cases} 
      1/S & o = o^*  \\
      0 & else
   \end{cases} 
$  
This is equivalent to an equation that implements the averaging in the denominator as a linear filter, that scales the filter aggregate produced by the sum over scales/orientations. This changes the dimensionality of the denominator, but not actually the result of the division, since image by image division is executed pixel-wise

$$ODOG: \frac{f_{o^*,s^*}}{\sqrt{F_{x,y}*(\sum_{o,s=0}^{5,6}{w_{o,s} f_{o,s})^2}}}$$
$w_{o,s}$ same as above,  
$F_{x,y} = \frac{1}{1024^2}$ everywhere.

By renaming the remaining sum in the denominator, we can find an abstract formulation of the RMS-Norm as a sequence of linear filters

 $$\frac{f_{o^*,s^*}}{\sqrt{F_{x,y}*(\sum_{o,s=0}^{5,6}{w_{o,s} f_{o,s})^2}}}=\frac{f_{o,s}}{\sqrt{G * N^2}} = \frac{f_{o,s}}{\sqrt{G * (W \cdot f)^2}}$$
 
 Which helps us develop terminology. The entire fraction is called the divisive norm on the filterbank-output $f$.  
 $G * N^2$ is the local energy, calculated as the convolution of $G$ the spatial weighting, and $N^2$ the Normcoefficient image squared. The Normcoefficient Image can be calculated in turn by computing the Normpool $W$ dotproduct against the same $f$ that we are normalizing.
   
An implementation of this formulation can be found here:

In [ ]:
def divisive_norm(filter_outputs, o, s):
    G = spatial_weighting(filter_outputs.shape[-2:])
    N = normcoeff(filter_outputs, o, s)**2
    
    z = scipy.signal.convolve(G, N, mode="same")
    normed_filter = filter_outputs[o,s,:] / np.sqrt(z)
    return normed_filter

def normcoeff(filter_outputs, o_0, s_0): 
    w = normpool(filter_outputs.shape[0],filter_outputs.shape[1], o_0, s_0)
    coeffs = np.tensordot(filter_outputs, w, axes=([0, 1], [0, 1]))
    return coeffs

def normpool(S, O, o_0, s_0): # w_{o,s}
    ODOG = True
    LODOG, FLODOG = False, False
    w = np.ones(shape=(S,O))
    for o, s in zip(range(O), range(S)):
        if ODOG or LODOG:
            if o==o_0:
                w[o,s] = 1/S
            else:
                w[o,s] = 0

        if FLODOG:
            if o==o_0:
                w[o,s] = 1 #gaussian(s-s_0)
            else:
                w[o,s] = 0
    return w

def spatial_weighting(filter_shape):
    ODOG = True
    LODOG, FLODOG = False, False
    if ODOG:
        G = np.ones(filter_shape) / np.prod(filter_shape)
        
    if LODOG:
        G = 1 #gaussian(filter_shape, sigma)

    if FLODOG:
        G = 1 #gaussian(filter_shape, sigma=k*s)
    return G

Our goal is now to establish numerical equivalence between the traditional formulation and the result of the newly reformatted and mathematically motivated code

The original implementation is already tested against the matlab implementation of 